In [2]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [3]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.01
sim_data = get_sim_data(x0,env,Δt,true);

In [8]:
q0 = [1., 0., 0., 0., 0., 0., .0]
v0 = [0., 0., 0., 2., 0., 0.]
u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]

z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))

τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=true);
τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false);

Solve_Succeeded


In [9]:
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)
display("Explicit")
display(traj[sim_data.num_q+sim_data.num_v+1:end,2])

"IP"

7-element Array{Float64,1}:
 0.0       
 7.22901e-9
 2.44494e-8
 0.5       
 2.44494e-8
 1.95102   
 1.0       

"Aug Lag"

7-element Array{Float64,1}:
  0.0      
  0.0286656
 -0.0049155
  0.2065   
  0.100489 
  2.65868  
  0.757053 

"Explicit"

7-element Array{Float64,1}:
  0.0       
 -3.6374e-9 
 -1.70884e-9
  0.5       
 -1.70884e-9
  1.95095   
  1.0       

In [10]:
display("IP")
display(τ_ip)
display("Aug Lag")
display(τ_auglag)

"IP"

6-element Array{Float64,1}:
  0.0        
  0.0        
  5.11763e-63
  4.905      
 -6.0069e-16 
 -9.81001    

"Aug Lag"

6-element Array{Float64,1}:
  2.2668e-26 
  0.0        
  3.87432e-63
  1.74456    
  1.03402    
 -7.42669    

In [24]:
ForwardDiff.jacobian(x̃ -> solve_implicit_contact_τ(sim_data,x̃[1:sim_data.num_contacts],Dtv,HΔv,dyn_bias,u0,ip_method=false)[1],rand(sim_data.num_contacts))

6×1 Array{Float64,2}:
 -8.3312e-27 
  0.0        
  2.51503e-27
 -1.22651    
  0.0144872  
  5.73544    

In [ ]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [ ]:
N = size(traj,2)
qs = [traj[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .25);